In [30]:
# calc NMI scores
# continue from 
# http://localhost:8888/notebooks/66%20CausalMap/Panasonic-IDS/notebooks/EDA_Check%20Clustering.ipynb

import networkx as nx
import os
import re
import pandas as pd
import numpy as np

data = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR_annotated_adjusted_w_scores.csv")
data = data.drop(columns=['cause','effect']).rename(columns={'cause_wi':'cause','effect_wi':'effect'})
ref = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR_spantopics.csv")
ref['topic- Debdeep'] = ref['topic- Debdeep'].apply(lambda x: str(x).lower())
ref2 = pd.read_csv(os.path.join(r'D:\66 CausalMap\Panasonic-IDS\data','MIR_annotated_adjusted_w_scores.csv'))
ref = pd.pivot_table(ref, values='topic- Debdeep', index=['index'], columns=['ce_label'], aggfunc=lambda x: ' '.join(x)).reset_index()
ref = ref.merge(ref2[['text_w_pairs','index']], how='left', on='index')
ref

,index,cause,effect,text_w_pairs
0,MIR_143_1_0,low delivery time,risk mitigation,The Japanese automaker pioneered so-called Jus...
1,MIR_143_3_0,process improvement,innovation,<ARG0>From fashion to food processing to pharm...
2,MIR_143_3_1,strategy,process improvement,<ARG1>From fashion to food processing to pharm...
3,MIR_143_4_0,business hurdles,risk,But the tumultuous events of the past year hav...
4,MIR_143_5_0,transportation disruption,raw material shortage,<ARG0>As the pandemic has hampered factory ope...
5,MIR_1785_4_0,technology improvement,innovation,<ARG0>Technogoical advancements such us automa...
6,MIR_1785_4_1,innovation,obsolete components,Technogoical advancements such us automated co...
7,MIR_1785_5_0,green energy adoption,innovation,<ARG1>Implementation of automotive camless pis...
8,MIR_2060_8_0,product improvement,design innovation,The 2022 Maruti Suzuki Brezza will <ARG1>conti...
9,MIR_3225_4_0,talent,technology hub,<ARG1>Entering new locationsWe strategically c...


In [31]:
final = data.merge(ref, on='index', how='left', suffixes=['_model','_user'])
final = final.dropna(subset=['cause_model','effect_model'], how='any')
final

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs_model,seq_label,pair_label,context,...,token_overlap,row_num,source,support,evidence,cause_model,effect_model,cause_user,effect_user,text_w_pairs_user
0,MIR,51,1,0,MIR_51_1_0,"From to Uber and , many foreign companies have...","<ARG1>From to Uber and , many foreign companie...",1,1,NaN,...,1,16.0,f-51,1.0,in a bid to make in roads in the indian market...,17>>make_bid_sense_business_good,"54>>and , many foreign companies have had to l...",increased competition,price reduction,"<ARG1>From to Uber and , many foreign companie..."
1,MIR,51,3,0,MIR_51_3_0,Foreign or local electric cars (EVs) will grow...,<ARG1>Foreign or local electric cars (EVs) wil...,1,1,NaN,...,1,18.0,f-51,1.0,there ' s price par ity with its internal comb...,24>>electric_battery_swap_agreement_below,2>>or_cars__producing_passenger,cost reduction,increased demand,<ARG1>Foreign or local electric cars (EVs) wil...
2,MIR,51,6,0,MIR_51_6_0,"""Given these conditions, it is understandable ...","<ARG0>""Given these conditions</ARG0>, it is un...",1,1,NaN,...,1,19.0,f-51,1.0,"these conditions --> , it is understand able t...",47>>these conditions,"56>>, it is understand able that local as well...",competition,supply reduction,"<ARG0>""Given these conditions</ARG0>, it is un..."
3,MIR,78,0,0,MIR_78_0_0,Toyota in India has largely pivoted toward hyb...,<ARG1>Toyota in India has largely pivoted towa...,1,1,NaN,...,1,33.0,f-78,1.0,"they aren ��� t purely electric . --> , which ...",37>>they aren ’ t purely electric .,0>>taxes_as_much_compared_fold,hybrid,taxation,<ARG1>Toyota in India has largely pivoted towa...
4,MIR,78,2,0,MIR_78_2_0,The government keeps taxes on cars and motorbi...,<ARG0>The government keeps taxes on cars and m...,1,1,NaN,...,1,34.0,f-78,1.0,the government keeps taxes on cars and motor b...,0>>taxes_as_much_compared_fold,57>>that companies find it hard to build scale...,taxation,supply reduction,<ARG0>The government keeps taxes on cars and m...
5,MIR,78,8,0,MIR_78_8_0,"Elon Musk, the billionaire founder of Tesla In...","Elon Musk, the billionaire founder of Tesla In...",1,1,NaN,...,1,35.0,f-78,1.0,import duties would --> his vehicles un af for...,45>>import duties would,22>>his_divide_hit_af_ford,taxation,supply reduction,"Elon Musk, the billionaire founder of Tesla In..."
6,MIR,94,1,0,MIR_94_1_0,The report’s focus solely on electric vehicles...,The report’s focus solely on electric vehicles...,1,1,NaN,...,1,39.0,f-94,1.0,marks a shift away from the current policy tha...,24>>electric_battery_swap_agreement_below,1>>emissions_devices_engines_worrying_were,hybrid,uncertainly,The report’s focus solely on electric vehicles...
7,MIR,94,6,0,MIR_94_6_0,"To kick-start the shift, the report suggests b...","To kick-start the shift, the report suggests <...",1,1,NaN,...,1,40.0,f-94,1.0,to bring down their cost --> the report sugges...,4>>bring_cost_them_down_stock,24>>electric_battery_swap_agreement_below,incentives,cost reduction,"To kick-start the shift, the report suggests <..."
8,MIR,94,7,0,MIR_94_7_0,It also recommends setting up battery swapping...,It also recommends <ARG1>setting up battery sw...,1,1,NaN,...,1,41.0,f-94,1.0,to bring them to cost par ity with conventiona...,4>>bring_cost_them_down_stock,24>>electric_battery_swap_agreement_below,cost reduction,improved infrastructure,It also recommends <ARG1>setting up battery sw...
9,MIR,143,1,0,MIR_143_1_0,The Japanese automaker pioneered so-called Jus...,The Japanese automaker pioneered so-called Jus...,1,1,NaN,...,1,0.0,f-143,1.0,", in which parts are delivered to factories ri...",13>>want_manufactures_meet_suppliers_are,4>>bring_cost_them_down_stock,low delivery time,risk mitigation,The Japanese automaker pioneered so-called Jus...


In [32]:
from sklearn.metrics.cluster import normalized_mutual_info_score

users = list(final['cause_user'])+list(final['effect_user'])
models = list(final['cause_model'])+list(final['effect_model'])
normalized_mutual_info_score(users, models)

0.8983732470555363

In [34]:
sorted([f'{a}__{b}' for a,b in zip(users,models)])

[' business hurdles__31>>, while re in vi gor ating concerns that some industries have gone too far',
 "accident__35>>[] , 54 , was killed in a car crash in [] ' s [] l gh ar district near []",
 'audit__27>>suzuki_week_offices_raided_secutor',
 'audit__27>>suzuki_week_offices_raided_secutor',
 'audit__7>>safety_improve_road_audit_conduct',
 'better economy__18>>economy_year_hu_ir_cular',
 'competition__47>>these conditions',
 'compliance__1>>emissions_devices_engines_worrying_were',
 'compliance__1>>emissions_devices_engines_worrying_were',
 'cost increase__14>>automotive_among_major_industry_shift',
 'cost reduction__24>>electric_battery_swap_agreement_below',
 'cost reduction__24>>electric_battery_swap_agreement_below',
 'cost reduction__4>>bring_cost_them_down_stock',
 'cost reduction__50>>and has recently offered incentives including tax breaks and subsidies',
 'design innovation__51>>the 202 2 [] will continue with its global c platform but will receive structural updates for an i

In [35]:
sorted([f'{b}__{a}' for a,b in zip(users,models)])

['0>>taxes_as_much_compared_fold__taxation',
 '0>>taxes_as_much_compared_fold__taxation',
 '10>>chip_shortage_global_on_by__supply shortage',
 '10>>chip_shortage_global_on_by__supply shortage',
 '11>>model_range_advance_electrified_its__innovation',
 '12>>french_fur_lough_deal_the__supply shortage',
 '13>>want_manufactures_meet_suppliers_are__low delivery time',
 '14>>automotive_among_major_industry_shift__cost increase',
 '14>>automotive_among_major_industry_shift__innovation',
 '15>>advanced_automated_control_such_systems__technology improvement',
 '17>>make_bid_sense_business_good__increased competition',
 '18>>economy_year_hu_ir_cular__better economy',
 '1>>emissions_devices_engines_worrying_were__compliance',
 '1>>emissions_devices_engines_worrying_were__compliance',
 '1>>emissions_devices_engines_worrying_were__fraud',
 '1>>emissions_devices_engines_worrying_were__safety violation',
 '1>>emissions_devices_engines_worrying_were__uncertainly',
 '21>>order to be better prepared for 

In [36]:
from collections import Counter
print(len(Counter(users)))
print(len(Counter(models)))

50
51


In [22]:
import networkx as nx
import os
import re
import pandas as pd
import numpy as np

In [24]:
node_weights = users
node_weights = dict(Counter(node_weights))

G = nx.DiGraph()

for node, weight in node_weights.items():
    G.add_node(node.split('>>')[-1], weight=weight)

for i,row in final.iterrows():
    G.add_edge(
        row.cause_user.split('>>')[-1], 
        row.effect_user.split('>>')[-1], 
        weight=1,
    )
    
graph_folder = r"D:\66 CausalMap\SciLit_CausalMap\visualization\mir_paper_test"
nx.write_gml(G, os.path.join(graph_folder, "overlap_user_debdeep.gml"))

In [26]:
node_weights = models
node_weights = dict(Counter(node_weights))

G = nx.DiGraph()

for node, weight in node_weights.items():
    G.add_node(node.split('>>')[-1], weight=weight)

for i,row in final.iterrows():
    G.add_edge(
        row.cause_model.split('>>')[-1], 
        row.effect_model.split('>>')[-1], 
        weight=1,
    )
    
graph_folder = r"D:\66 CausalMap\SciLit_CausalMap\visualization\mir_paper_test"
nx.write_gml(G, os.path.join(graph_folder, "overlap_model.gml"))